# Congestion Charges - Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app18-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@64447db

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@4c5be1c6

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val camera = hiveCxt.table("sqlzoo.camera")
val keeper = hiveCxt.table("sqlzoo.keeper")
val vehicle = hiveCxt.table("sqlzoo.vehicle")
val image = hiveCxt.table("sqlzoo.image")
val permit = hiveCxt.table("sqlzoo.permit")

camera: DataFrame = [id: int, perim: string]
keeper: DataFrame = [id: int, name: string ... 1 more field]
vehicle: DataFrame = [id: string, keeper: int]
image: DataFrame = [camera: int, whn: string ... 1 more field]
permit: DataFrame = [reg: string, sdate: string ... 1 more field]

## 1.
List the owners (name and address) of Vehicles caught by camera 1 or 18 without duplication.

In [5]:
(keeper.withColumnRenamed("id", "keeper")
 .join(vehicle, "keeper")
 .join(image.filter(image("camera").isin(List(1, 18): _*)), 
       (vehicle("id") === image("reg")))
 .select("name", "address")
 .distinct()
 .showHTML())

name,address
"Ambiguous, Arthur",Absorption Ave.
"Inconspicuous, Iain",Interception Rd.
"Strenuous, Sam",Surjection Street


## 2.
Show keepers (name and address) who have more than 5 vehicles.

In [6]:
(keeper.withColumnRenamed("id", "keeper")
 .join(vehicle, "keeper")
 .groupBy("name", "address")
 .agg(count("keeper").alias("keeper"))
 .filter(col("keeper")>5)
 .select("name", "address")
 .showHTML())

name,address
"Ambiguous, Arthur",Absorption Ave.
"Inconspicuous, Iain",Interception Rd.


## 3.
For each vehicle show the number of current permits (suppose today is the 1st of Feb 2007). The list should include the vehicle.s registration and the number of permits. Current permits can be determined based on charge types, e.g. for weekly permit you can use the date after 24 Jan 2007 and before 02 Feb 2007.

In [7]:
(permit.withColumn("sdate", to_timestamp(col("sdate")))
 .withColumn("edate", when(
    col("chargetype") === "Daily", col("sdate") + expr("interval 1 day")).when(
    col("chargetype") === "Weekly", col("sdate") + expr("interval 1 week")).when(
    col("chargetype") === "Monthly", col("sdate") + expr("interval 1 month")).when(
    col("chargetype") === "Annual", col("sdate") + expr("interval 1 year")))
 .filter((col("sdate") <= "2007-02-01") && (col("edate") >= "2007-02-01"))
 .groupBy("reg")
 .agg(count("chargetype").alias("reg"))
 .orderBy("reg")
 .showHTML())

reg,reg
SO 02 DSP,1
SO 02 DTP,1
SO 02 JSP,1
SO 02 KSP,1
SO 02 KTP,1
SO 02 QSP,1
SO 02 RSP,1


## 4.
Obtain a list of every vehicle passing camera 10 on 25th Feb 2007. Show the time, the registration and the name of the keeper if available.

In [8]:
(vehicle.join(keeper.withColumnRenamed("id", "keeper"), "keeper")
 .join(image.filter((image("whn").between("2007-02-25", "2007-02-26")) &&
                    (image("camera") === 10)), (vehicle("id") === image("reg")))
 .select("reg", "whn", "name")
 .showHTML())

reg,whn,name
SO 02 CSP,2007-02-25 07:45:11.0,"Ambiguous, Arthur"
SO 02 ESP,2007-02-25 18:08:40.0,"Ambiguous, Arthur"


## 5.
List the keepers who have more than 4 vehicles and one of them must have more than 2 permits. The list should include the names and the number of vehicles.

In [9]:
// keepers who have more than 2 permits
val t = (vehicle
          .join(permit
          .groupBy("reg")
          .agg(count("chargetype").alias("npermit"))
          .filter(col("npermit") > 2)
          .select("reg").alias("p"), 
          (vehicle("id") === col("reg")))
    .select("keeper"))

(vehicle.join(keeper.withColumnRenamed("id", "keeper")
              .join(t, "keeper"),
              "keeper")
 .groupBy("name")
 .agg(count("id").alias("n_vehicle"))
 .select("name", "n_vehicle")
 .filter(col("n_vehicle")>4)
 .showHTML())

name,n_vehicle
"Inconspicuous, Iain",7


t: DataFrame = [keeper: int]

In [10]:
spark.stop()